In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv(r'train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [3]:
x_input = df_train.drop(columns=['label']).values
x_aux = x_input.reshape(-1, 28, 28)
v = np.sum(x_aux, axis=1)
h = np.sum(x_aux, axis=2)

In [4]:
from sklearn.preprocessing import StandardScaler
v_scaler = StandardScaler()
v_scaled = v_scaler.fit_transform(v)

h_scaler = StandardScaler()
h_scaled = h_scaler.fit_transform(h)

In [5]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
x_scaled = mm_scaler.fit_transform(x_input)

In [10]:
x = np.hstack([x_scaled, v_scaled, h_scaled])
y = pd.get_dummies(df_train['label'])

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [12]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout

model = Sequential()

model.add(Dense(128, activation='tanh', input_shape=(x_train.shape[1], ), kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])

In [13]:
model.fit(x_train, y_train, batch_size=16, epochs = 20, verbose=1, validation_data=(x_test, y_test))

Epoch 1/20
2100/2100 [==============================] - 7s 3ms/step - loss: 0.3651 - accuracy: 0.8904 - categorical_accuracy: 0.8904 - val_loss: 0.2262 - val_accuracy: 0.9351 - val_categorical_accuracy: 0.9351
Epoch 2/20
2100/2100 [==============================] - 6s 3ms/step - loss: 0.2163 - accuracy: 0.9349 - categorical_accuracy: 0.9349 - val_loss: 0.1758 - val_accuracy: 0.9494 - val_categorical_accuracy: 0.9494
Epoch 3/20
2100/2100 [==============================] - 6s 3ms/step - loss: 0.1807 - accuracy: 0.9455 - categorical_accuracy: 0.9455 - val_loss: 0.1699 - val_accuracy: 0.9499 - val_categorical_accuracy: 0.9499
Epoch 4/20
2100/2100 [==============================] - 6s 3ms/step - loss: 0.1584 - accuracy: 0.9514 - categorical_accuracy: 0.9514 - val_loss: 0.1422 - val_accuracy: 0.9588 - val_categorical_accuracy: 0.9588
Epoch 5/20
2100/2100 [==============================] - 6s 3ms/step - loss: 0.1396 - accuracy: 0.9574 - categorical_accuracy: 0.9574 - val_loss: 0.1298 - val_ac

In [17]:
df_test = pd.read_csv('test.csv')
x_out = mm_scaler.transform(df_test.values)
x_out_aux = x_out.reshape(-1, 28, 28)
v_out = v_scaler.transform(np.sum(x_out_aux, axis=1))
h_out = h_scaler.transform(np.sum(x_out_aux, axis=2))
x_output = np.hstack([x_out, v_out, h_out])
y_pred = np.argmax(model.predict(x_output), axis=1).reshape(-1, 1)

875/875 [==============================] - 2s 2ms/step


In [18]:
df = pd.DataFrame(y_pred, columns=['Label'])
df.index += 1
df = df.rename_axis('ImageId')
print(df.head)

<bound method NDFrame.head of          Label
ImageId       
1            2
2            0
3            9
4            0
5            3
...        ...
27996        9
27997        7
27998        3
27999        4
28000        2

[28000 rows x 1 columns]>


In [19]:
df.to_csv('submission.csv')